# Practice for Azure AI and LUIS

20220415 練習
我是一家 XXX 手搖飲料店的老闆，最近我們想要引入語音點餐服務，
想麻煩AI班的同學使用Azure AI 服務，將我們的需求流程完成

流程如下：
語音轉字串給LUIS，客戶透過語音的方式點餐，
EX：I want a bottle of milk tea.
透過Azure語音紀錄點餐內容(印出)
機器人自動回覆(語音)
語音：Your milk tea has already been ordered.

In [1]:
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
import datetime, json, os, time
import azure.cognitiveservices.speech as speechsdk

In [2]:
speech_key, service_region = "a0453824b284450abc1c5508f98e7dc0", "eastus"
luisAppID = '9fbd4931-2a0f-42df-8be8-da0bb3ed7f5e'
runtime_key = 'd688ec74663c49c19a472b9719ca7c94'
runtime_endpoint = 'https://luis0415-authoring.cognitiveservices.azure.com/'

In [3]:
#將語音轉成文字(英文)
def speech_to_text():

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)
    print("Say something...")
    result = speech_recognizer.recognize_once()

    # Check the result
    if result.reason == speechsdk.ResultReason.TranslatedSpeech:
        print("RECOGNIZED '{}': {}".format(fromLanguage, result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("NOMATCH: Speech could not be recognized: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        print("CANCELED: Reason={}".format(result.cancellation_details.reason))
        if result.cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("CANCELED: ErrorDetails={}".format(result.cancellation_details.error_details))
    return result.text



In [4]:
#使用LUIS抓取關鍵字
# production or staging
luisSlotName = 'production'

# Instantiate a LUIS runtime client
clientRuntime = LUISRuntimeClient(runtime_endpoint, CognitiveServicesCredentials(runtime_key))

def predict(app_id, slot_name):

    request = { "query" : speech_to_text() }
    response = clientRuntime.prediction.get_slot_prediction(app_id=app_id, slot_name=slot_name, \
                                                            prediction_request=request)

    if response.prediction.entities == {}:
        print("Sorry, we don't have this drink.")
        print("Please try other drink.")
        return(predict(luisAppID, luisSlotName))
    else:
        return (response.prediction.entities['facet'][0])
    


In [6]:
#將文字轉回語音
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_recognition = "zh-TW"
# Creates a speech synthesizer using the default speaker as audio output.
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# Receives a text from console input.
print("What do you want to drink today? ")
text = "Your {} has already been ordered.".format(predict(luisAppID, luisSlotName))


result = speech_synthesizer.speak_text_async(text).get()

#Checks result.
if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized to speaker for text [{}]".format(text))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
    print("Did you update the subscription info?")

What do you want to drink today? 
Say something...
Sorry, we don't have this drink.
Please try other drink.
Say something...
Sorry, we don't have this drink.
Please try other drink.
Say something...
Speech synthesized to speaker for text [Your Coffee has already been ordered.]
